In [47]:
import pandas as pd
from collections import Counter
from imblearn.over_sampling import SMOTE 
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
import numpy as np

In [48]:
df = pd.read_csv("data_details/trialPromoResults_onehot.csv")

In [49]:
df = df.drop(["index"], axis=1)
print('Original dataset shape {}'.format(Counter(y)))

Original dataset shape Counter({'None': 828, 'A': 133, 'B': 39})


In [50]:
df.head()
headers = df.columns.values

### Insert the data into train and test 

In [51]:
train_df, test_df = train_test_split(df, test_size=0.25, random_state=92)
train_df.head()

,F,M,married,widowed,single,divorced,age,children,construct,education,...,medicine,retired,secondary,tertiary,postgrad,professional,income,avbal,avtrans,decision
988,1,0,1,0,0,0,51.40,2,0,0,...,0,0,0,0,1,0,2692.48,20954.73,2516.79,None
990,0,1,0,0,1,0,20.50,0,1,0,...,0,0,1,0,0,0,2692.48,20954.73,2516.79,None
972,1,0,1,0,0,0,52.02,0,0,0,...,0,0,1,0,0,0,889.98,19738.64,1201.90,None
989,0,1,1,0,0,0,25.79,0,1,0,...,0,0,1,0,0,0,2692.48,20954.73,2516.79,A
23,0,1,0,0,1,0,27.43,0,0,1,...,0,0,0,1,0,0,4121.55,19983.49,1102.11,None


In [52]:
test_df.head()

,F,M,married,widowed,single,divorced,age,children,construct,education,...,medicine,retired,secondary,tertiary,postgrad,professional,income,avbal,avtrans,decision
92,1,0,1,0,0,0,66.90,0,0,0,...,0,1,1,0,0,0,1927.01,5491.91,815.74,None
209,0,1,0,0,1,0,27.69,0,0,0,...,0,0,1,0,0,0,2082.23,17948.54,982.88,None
817,1,0,0,0,0,1,53.72,2,0,0,...,0,0,0,1,0,0,6183.15,25756.82,730.38,None
130,1,0,0,0,1,0,28.31,0,0,0,...,0,0,0,1,0,0,7067.84,14085.66,3020.80,None
995,1,0,0,0,0,1,47.56,0,0,0,...,0,0,0,0,0,1,11143.41,17825.67,3742.27,None


In [53]:
X_train = train_df.loc[:, train_df.columns != "decision"]
y_train = train_df["decision"]
X_train.head()

,F,M,married,widowed,single,divorced,age,children,construct,education,...,manuf,medicine,retired,secondary,tertiary,postgrad,professional,income,avbal,avtrans
988,1,0,1,0,0,0,51.40,2,0,0,...,1,0,0,0,0,1,0,2692.48,20954.73,2516.79
990,0,1,0,0,1,0,20.50,0,1,0,...,0,0,0,1,0,0,0,2692.48,20954.73,2516.79
972,1,0,1,0,0,0,52.02,0,0,0,...,0,0,0,1,0,0,0,889.98,19738.64,1201.90
989,0,1,1,0,0,0,25.79,0,1,0,...,0,0,0,1,0,0,0,2692.48,20954.73,2516.79
23,0,1,0,0,1,0,27.43,0,0,1,...,0,0,0,0,1,0,0,4121.55,19983.49,1102.11


### Run the SMOTE algorithm to balance the output classes as mentioned [here](http://contrib.scikit-learn.org/imbalanced-learn/stable/generated/imblearn.over_sampling.SMOTE.html)

In [54]:
sm = SMOTE(random_state=42)
X_res, y_res = sm.fit_sample(X_train, y_train)

In [55]:
print('Resampled dataset shape {}'.format(Counter(y_res)))

Resampled dataset shape Counter({'None': 621, 'A': 621, 'B': 621})


### So SMOTE in Python cannot yet handle categorical data - refer [this](https://stackoverflow.com/questions/47655813/oversampling-smote-for-binary-and-categorical-data-in-python)

In [56]:
X_res

array([[1.00000000e+00, 0.00000000e+00, 1.00000000e+00, ...,
        2.69248000e+03, 2.09547300e+04, 2.51679000e+03],
       [0.00000000e+00, 1.00000000e+00, 0.00000000e+00, ...,
        2.69248000e+03, 2.09547300e+04, 2.51679000e+03],
       [1.00000000e+00, 0.00000000e+00, 1.00000000e+00, ...,
        8.89980000e+02, 1.97386400e+04, 1.20190000e+03],
       ...,
       [8.94479931e-01, 1.05520069e-01, 0.00000000e+00, ...,
        1.90025507e+03, 1.67555379e+04, 5.73772250e+02],
       [9.91475335e-01, 8.52466485e-03, 0.00000000e+00, ...,
        4.45949750e+03, 1.74640340e+04, 3.82894098e+03],
       [6.15170262e-02, 9.38482974e-01, 0.00000000e+00, ...,
        1.39235299e+04, 2.97638223e+04, 3.19203978e+03]])

In [57]:
columns_to_not_round = [6, 21, 22, 23]
sampled_data = ",".join(headers) + "\n" 
    
for itr in range(0, len(X_res)):
    for jtr in range(0, len(X_res[0])):
        if jtr not in columns_to_not_round:
            sampled_data += str(round(X_res[itr][jtr])) + ","
        else:
            sampled_data += str(X_res[itr][jtr]) + ","
    sampled_data += str(y_res[itr]) + "\n"
    
with open("data_details/trial_promo_training.csv", "w") as fw:
    fw.writelines(sampled_data)
    

In [58]:
test_df.to_csv("data_details/trial_promo_testing.csv", index = False)